In [193]:
import os
import numpy as np
from csv import reader
from config import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
selected = []
with open(metadata_path) as f:
    for line in reader(f):
        if line[-1] == 'TRUE':
            selected.append(line)

In [195]:
len(selected) 
# the selected criterior is that it needs to have > 100 samples and corresponding roadmap category
#17 samples selected

17

In [196]:
for item in selected:
    print(item[0][:-3])
#files that we need to use, I added to config.py

Adipose_Subcutaneous.allpairs.txt
Adipose_Visceral_Omentum.allpairs.txt
Artery_Aorta.allpairs.txt
Breast_Mammary_Tissue.allpairs.txt
Cells_EBV-transformed_lymphocytes.allpairs.txt
Colon_Sigmoid.allpairs.txt
Colon_Transverse.allpairs.txt
Esophagus_Gastroesophageal_Junction.allpairs.txt
Esophagus_Mucosa.allpairs.txt
Esophagus_Muscularis.allpairs.txt
Heart_Atrial_Appendage.allpairs.txt
Heart_Left_Ventricle.allpairs.txt
Lung.allpairs.txt
Muscle_Skeletal.allpairs.txt
Pancreas.allpairs.txt
Stomach.allpairs.txt
Whole_Blood.allpairs.txt


In [197]:
roadmap_dict = {}
for item in selected:
    roadmap_dict[item[0][:-3]] = item[2]

In [198]:
file_idx = 0 # which assiciation file to look at
association_file = os.path.join(association_path, files[file_idx])
f = open(association_file)
header = f.readline().strip().split()
line = f.readline().strip().split()

print(header)

['gene_id', 'variant_id', 'tss_distance', 'ma_samples', 'ma_count', 'maf', 'pval_nominal', 'slope', 'slope_se']


In [199]:
def get_expression_matrix(filename):
    with open(filename) as f:
        lines = f.readlines()
        head = lines[2]
        head = head.strip().split('\t')[2:]
        lines = lines[3:]

        expression_matrix = np.zeros((len(head), len(lines)))
        names = []
        for i, line in enumerate(lines):
            ls = line.strip().split('\t')
            names.append(ls[0])
            ls = np.array(list(map(float, ls[2:])))
            expression_matrix[:,i] = ls
            
        return expression_matrix, head, names
expression_matrix, head, gene_name = get_expression_matrix(os.path.join(utils_path, 'GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct'))
print(expression_matrix.shape, len(head), len(gene_name))
gene_idx = dict(zip(gene_name, range(len(gene_name))))
tissue_idx = head.index(files[file_idx].split('.')[0] .replace('_', ' - '))

(53, 56202) 53 56202


In [200]:
def process_line(line):
    gene = line[0]
    variant = line[1]
    pval = float(line[6])
    chromosome = variant.split('_')[0]
    dist = int(line[2])
    return gene, variant, pval, chromosome, dist

In [201]:
def get_gene_features(gene, expression_matrix, gene_idx):
    # get features of a gene
    expression = expression_matrix[gene_idx[gene]]
    return expression

In [202]:
snp_feat_dict = {}
with open(os.path.join(utils_path, 'snp_feat.txt')) as f:
    for line in f:
        ls = line.strip().split(',')
        snp_feat_dict[ls[0]] = ls[-1]

KeyboardInterrupt: 

In [116]:
def get_snp_features(snp):
    # get features of a snp
    return snp_feat_dict.get(snp, 'NA')

In [124]:
tss_gene_dict = {}
with open(os.path.join(utils_path, 'tss.txt')) as f:
    ln = 0
    for line in f:
        if ln > 0:
            ls = line.strip().split(',')
            tss_gene_dict[ls[0]] = int(ls[-1])
        ln += 1

In [150]:
def get_pair_features(gene, snp):
    # get features of a gene-snp pair
    #print(gene)
    if '.' in gene:
        gene = gene.split('.')[0]
        
    if gene in tss_gene_dict:
        gene_pos = tss_gene_dict[gene]
    else:
        gene_pos = np.nan
    
    snp_pos = int(snp.split('_')[1])
    return  snp_pos - gene_pos

In [190]:
roadmap_file = roadmap_dict[files[file_idx]]
#print(roadmap_file)

roadmap_15state_fn = os.path.join(data_path, 'roadmap', roadmap_file + '_15_coreMarks_dense.bed')
roadmap_25state_fn = os.path.join(data_path, 'roadmap', roadmap_file + '_25_imputed12marks_dense.bed')

chrom_size = {}
with open(os.path.join(utils_path, 'hg19.chrom.sizes')) as f:
    for line in f:
        ls = line.strip().split('\t')
        chrom_size[ls[0]] = int(ls[1])

roadmap_15state = {}
ln = 0
with open(roadmap_15state_fn) as f:
    for line in f:
        if ln > 0:
            ls = line.strip().split('\t')
            if not ls[0] in roadmap_15state:
                roadmap_15state[ls[0]] = -1 * np.ones((chrom_size[ls[0]] // 100)).astype(np.int)
            
            roadmap_15state[ls[0]][int(ls[1])//100 : int(ls[2])//100] = int(ls[3].split('_')[0])
            
        ln += 1
        if ln % 100000 == 0:
            print(ln)

            
roadmap_25state = {}
ln = 0
with open(roadmap_25state_fn) as f:
    for line in f:
        if ln > 0:
            ls = line.strip().split('\t')
            if not ls[0] in roadmap_25state:
                roadmap_25state[ls[0]] = -1 * np.ones((chrom_size[ls[0]] // 100)).astype(np.int)
            
            roadmap_25state[ls[0]][int(ls[1]) //100 : int(ls[2]) // 100] = int(ls[3].split('_')[0])
            
        ln += 1
        if ln % 100000 == 0:
            print(ln)

100000
200000
300000
400000
500000
100000
200000
300000
400000
500000
600000
700000
800000


In [186]:
def get_chromatin_state(chr_no, pos):
    if not 'chr' in chr_no:
        chr_no = 'chr{}'.format(chr_no)
    pos = pos // 100
    state_15 = roadmap_15state[chr_no][pos]
    state_25 = roadmap_25state[chr_no][pos]
    return state_15, state_25

In [187]:
def get_features_for_one_line(line):
    gene, variant, pval, chromosomem, tss_dist = process_line(line)
    expression = get_gene_features(gene, expression_matrix[tissue_idx], gene_idx)
    maf = get_snp_features(variant)
    
    if maf == 'NA':
        maf = 'nan'
        
    maf = float(maf)
    
    tss_dist2 = get_pair_features(gene, variant) #the tss_dist we calculated is 17 away from their annotation, so I
    #choose to use the tss_dist they provided
    #print(tss_dist, tss_dist2)
    
    vs =  variant.split('_')
    chrom_no = vs[0]
    pos = int(vs[1])
    
    
    chrom_state = get_chromatin_state(chrom_no, pos)
    return expression, maf, tss_dist, chrom_state[0], chrom_state[1], pval
    

In [188]:
len(snp_feat_dict)

31563503

In [192]:
f = open(association_file)
header = f.readline().strip().split()
#print(header)

for i in range(3000):
    line = f.readline().strip().split()
    print(get_features_for_one_line(line))

(11.85, nan, -16136, 5, 25, 0.00908288)
(11.85, nan, -11994, 5, 25, 0.156116)
(11.85, nan, 24868, 15, 25, 0.636433)
(11.85, 0.09605, 24937, 15, 25, 0.439837)
(11.85, 0.01158, 32367, 15, 25, 0.528716)
(11.85, 0.01038, 35096, 15, 25, 0.62121)
(11.85, 0.02756, 86193, 15, 25, 0.0122073)
(11.85, nan, 103607, 15, 25, 0.293888)
(11.85, 0.02796, 105650, 15, 25, 0.00882831)
(11.85, nan, 201341, 15, 25, 0.912713)
(11.85, nan, 201600, 15, 25, 0.442554)
(11.85, 0.2091, 203920, 15, 25, 0.0800819)
(11.85, nan, 204033, 15, 25, 0.957386)
(11.85, 0.1318, 497183, 15, 25, 0.995124)
(11.85, nan, 497240, 15, 25, 0.245769)
(11.85, nan, 497485, 15, 25, 0.133409)
(11.85, 0.08806, 500229, 15, 25, 0.821555)
(11.85, 0.07907, 502456, 15, 25, 0.0498458)
(11.85, 0.02875, 502498, 15, 25, 0.243491)
(11.85, nan, 502524, 15, 25, 0.190303)
(11.85, 0.01737, 502772, 15, 25, 0.946547)
(11.85, nan, 504572, 15, 25, 0.745634)
(11.85, nan, 504681, 15, 25, 0.886097)
(11.85, nan, 504994, 15, 25, 0.384878)
(11.85, nan, 507637, 15

(11.85, 0.3001, 915011, 14, 25, 0.154103)
(11.85, 0.01258, 915297, 14, 25, 0.176394)
(11.85, 0.8113, 915543, 14, 25, 0.464413)
(11.85, 0.8682, 915558, 14, 25, 0.758117)
(11.85, 0.01757, 915796, 14, 25, 0.281395)
(11.85, 0.9607, 915921, 14, 25, 0.475846)
(11.85, 0.884, 916059, 14, 25, 0.523004)
(11.85, 0.03694, 916643, 14, 25, 0.56046)
(11.85, 0.03634, 917446, 14, 25, 0.550888)
(11.85, 0.8257, 917481, 14, 25, 0.418524)
(11.85, 0.7965, 917985, 14, 25, 0.613915)
(11.85, 0.02616, 918225, 14, 25, 0.706468)
(11.85, 0.01478, 918256, 14, 25, 0.218539)
(11.85, 0.8157, 919139, 2, 1, 0.542686)
(11.85, nan, 919293, 2, 1, 0.609375)
(11.85, 0.903, 919317, 2, 1, 0.632611)
(11.85, 0.9032, 919368, 2, 1, 0.632611)
(11.85, 0.03594, 919500, 2, 3, 0.771222)
(11.85, 0.8998, 919682, 2, 3, 0.573674)
(11.85, 0.3389, 920055, 2, 3, 0.42538)
(11.85, 0.8257, 920101, 2, 3, 0.778445)
(11.85, 0.8728, 920372, 2, 2, 0.597062)
(11.85, 0.5264, 920560, 2, 2, 0.476779)
(11.85, 0.3271, 920690, 2, 14, 0.221724)
(11.85, 0.533

(0.1625, 0.04473, 726136, 13, 24, 0.163895)
(0.1625, 0.04153, 726726, 12, 23, 0.192305)
(0.1625, 0.1659, 726818, 12, 23, 0.464279)
(0.1625, 0.04073, 726819, 12, 23, 0.192305)
(0.1625, 0.3508, 726876, 12, 23, 0.549955)
(0.1625, 0.3508, 726885, 12, 23, 0.549955)
(0.1625, 0.1825, 727106, 11, 23, 0.608798)
(0.1625, 0.02496, 727129, 11, 23, 0.557907)
(0.1625, 0.1466, 727213, 12, 23, 0.881232)
(0.1625, 0.5034, 727253, 12, 23, 0.720423)
(0.1625, 0.02117, 727290, 12, 23, 0.286742)
(0.1625, 0.004792, 727334, 12, 23, 0.0266692)
(0.1625, 0.02416, 727953, 12, 23, 0.646081)
(0.1625, 0.2612, 727954, 12, 23, 0.548326)
(0.1625, 0.1536, 728220, 13, 24, 0.526671)
(0.1625, 0.04473, 728430, 13, 24, 0.272687)
(0.1625, 0.9543, 728505, 13, 24, 0.164284)
(0.1625, 0.1639, 728817, 13, 24, 0.959728)
(0.1625, 0.1691, 728828, 13, 24, 0.947387)
(0.1625, 0.5136, 729468, 12, 23, 0.888517)
(0.1625, 0.513, 729578, 11, 2, 0.914479)
(0.1625, 0.03954, 730112, 1, 22, 0.492538)
(0.1625, 0.5114, 731193, 11, 2, 0.825058)
(0.1

nan